## 0. Install Dependencies and Bring in Data

In [13]:
!pip install tensorflow pandas matplotlib sklearn

In [1]:
#install dependencies
import os
import pandas as pd #helps to read data in tabular form
import tensorflow as tf
import numpy as np

In [2]:
#checking
pip list

SyntaxError: invalid syntax (2460348229.py, line 2)

In [2]:
#checking
os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv','train.csv')

'jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv'

In [3]:
#load data
df = pd.read_csv(
    os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv','train.csv'))

In [4]:
#checking
df.head() #loads first few

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
#checking
df.tail() #loads last few

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [5]:
#checking
df.loc[7]['comment_text']

"Your vandalism to the Matt Shirvington article has been reverted.  Please don't do it again, or you will be banned."

In [9]:
#checking
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

## 1. Preprocess

In [6]:
from tensorflow.keras.layers import TextVectorization #import tv layer for tokenization--converts words to nos

In [7]:
#split data in comments and features
x = df['comment_text'] #stores comments inside x
y = df[df.columns[2:]].values #to get all the values--stores labels inside y--.values converts to numpy arr

In [8]:
#checking
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [9]:
MAX_FEATURES=200000 #how many words we wanna store in the vocab

In [10]:
#tokenizes--converts words to nos
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800, #max sentence length
                               output_mode='int') #maps every word to int

In [11]:
#trains to tokenize
vectorizer.adapt(x.values) #passes numpy arr to vectorizer

In [12]:
#checking
vectorizer('Hello World, life is great!')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([288, 263, 306,   9, 275], dtype=int64)>

In [13]:
vectorized_text = vectorizer(x.values) #tokenizes all words

In [14]:
#checking
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [16]:
#create tf data pipeline--mcsbp--map,cache,shuffle,batch,prefetch
#2 ways--from_tensor_slices or list_files
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) #halps bottlenecks

In [17]:
#checking
batch_x,batch_y = dataset.as_numpy_iterator().next() #grabs one batch

In [18]:
#checking
batch_y.shape

(16, 6)

In [19]:
#checking
len(dataset) #no. of batches

9974

In [20]:
#partitions
train = dataset.take(int(len(dataset)*.7)) #training on 70%
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) #validating on 20%
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1)) #testing on 10%

In [21]:
#checking
len(val)

1994

In [22]:
train.as_numpy_iterator().next() #takes next data

(array([[  3776,    463,    104, ...,      0,      0,      0],
        [   113,      4,     42, ...,      0,      0,      0],
        [    14,      9,     15, ...,      0,      0,      0],
        ...,
        [  1112,    635,    179, ...,      0,      0,      0],
        [114208,  10197,     37, ...,      0,      0,      0],
        [   136,   2835,      3, ...,      0,      0,      0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

## 2. Create Sequential Model

In [23]:
from tensorflow.keras.models import Sequential #import seq api model
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense , Embedding #layers to be used to build dl network

In [24]:
model = Sequential() #instantiate seq api
#1st cfeate embedding layer
model.add(Embedding(MAX_FEATURES+1,32)) #32 vals long each embedding
#creating bidir LSTM layer
model.add(Bidirectional(LSTM(32,activation='tanh'))) #32 features--bidir allows tp pass info in both dir--64 features
#3 feature extraction dense layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#final layer
model.add(Dense(6,activation='sigmoid')) #op will be bw 0 nd 1

In [25]:
#compiling
model.compile(loss='BinaryCrossentropy', optimizer='Adam') #BC loss--adam optimizer

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history = model.fit(train, epochs=1, validation_data=val) #training dnn--passing train dataset, specifying epoch=1(no of
#passes over entire training DS), passing validation data

 104/6981 [..............................] - ETA: 1:28:42 - loss: 0.2212

In [27]:
#checking
history.history

NameError: name 'history' is not defined

In [28]:
from matplotlib import pyplot as plt #to visualise loss matrix

In [29]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show() #no gplot as epoch=1

NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

## 3. Make Predictions

In [30]:
input_text = vectorizer('You are an idiot') #test texts thru vectorizer

In [31]:
#checking
input_text #vectorized ip text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 7, 20, 30, ...,  0,  0,  0], dtype=int64)>

In [32]:
#checking
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [33]:
batch = test.as_numpy_iterator().next() #grabs next batch

In [34]:
#checking
batch_X, batch_Y=test.as_numpy_iterator().next()

In [35]:
batch_Y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [73]:
#checking
#model.predict(np.expand_dims(input_text,0))
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 145ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [36]:
res = model.predict(np.expand_dims(input_text,0)) #prediction stored in res
#wrap in an numpy arr

1/1 [==============================] - 5s 5s/step


## 4. Evaluate Model

In [37]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy #import some metrics

In [38]:
#instantiate the metrics
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [39]:
#make predictions
for batch in test.as_numpy_iterator(): #loop thru every batch
    X_true, Y_true = batch #unpacking batch
    yhat = model.predict(X_true) #making prediction--passing comments
    
    Y_true = Y_true.flatten() #flatten predictions--so they r all in one array
    yhat = yhat.flatten()
    
    pre.update_state(Y_true, yhat) #updating each metrics
    re.update_state(Y_true, yhat)
    acc.update_state(Y_true, yhat)

1/1 [==============================] - 0s 320ms/step


In [ ]:
#printing
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}') 
#low as we trained for only one epoch--can be improved by increasing no of epochs--will take longer 

Precision: 0.020246857777237892, Recall:0.45074349641799927, Accuracy:0.0


## 5. Test and Gradio

In [42]:
pip install gradio jinja2

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip list

Package                      VersionNote: you may need to restart the kernel to use updated packages.

---------------------------- ---------
absl-py                      1.4.0
aiofiles                     23.1.0
aiohttp                      3.8.4
aiosignal                    1.3.1
altair                       5.0.1
anyio                        3.7.0
asttokens                    2.2.1
astunparse                   1.6.3
async-timeout                4.0.2
attrs                        23.1.0
backcall                     0.2.0
cachetools                   5.3.1
certifi                      2023.5.7
charset-normalizer           3.1.0
click                        8.1.3
colorama                     0.4.6
comm                         0.1.3
contourpy                    1.0.7
cycler                       0.11.0
debugpy                      1.6.7
decorator                    5.1.1
exceptiongroup               1.1.1
executing                    1.2.0
fastapi                      0.97.0
ffmpy      

In [43]:
#import sys
#sys.path.append(c:\users\shiri\appdata\roaming\python\python310\site-packages)
#import openai
import gradio as gr

In [4]:
pip install --upgrade pip

In [25]:
model.save('toxicity_model.h5') #saves the model as h5

NameError: name 'model' is not defined

In [44]:
model = tf.keras.models.load_model('toxicity_model.h5') #reload model

In [45]:
#test model
input_str = vectorizer('hey I freaking hate you!')

In [46]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 2s 2s/step


In [47]:
res

array([[0.96036947, 0.05602395, 0.6792587 , 0.0182177 , 0.5264267 ,
        0.06952823]], dtype=float32)

In [55]:
#implement func that's going to hook into the gradio
def score_comment(comment):
    vectorized_comment = vectorized([comment])
    results = model.predict(vectorized_comment)
    
    text = '' #unpack results
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    return text

In [59]:
#create gradio interface
interface = gr.Interface(fn=score_comment, inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'), outputs='text')

C:\Users\shiri\ToxicComments\toxic\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\shiri\ToxicComments\toxic\lib\site-packages\gradio\inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
C:\Users\shiri\ToxicComments\toxic\lib\site-packages\gradio\inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


In [58]:
interface.launch(share=True) #makes publicly available link

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://8eb6ba0ec0a1cf495c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\shiri\ToxicComments\toxic\lib\site-packages\gradio\routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\shiri\ToxicComments\toxic\lib\site-packages\gradio\blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "C:\Users\shiri\ToxicComments\toxic\lib\site-packages\gradio\blocks.py", line 1077, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\shiri\ToxicComments\toxic\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "C:\Users\shiri\ToxicComments\toxic\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "C:\Users\shiri\ToxicComments\toxic\lib\site-packages\anyio\_backends\_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "C:\Users\shiri\AppData\Local\Temp\ipyke